In [1]:
# LSTM for international airline passengers problem with window regression framing
import numpy
import numpy as np
import keras
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from keras.utils.vis_utils import plot_model

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed 

In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [3]:
    def create_dataset2(dataset, look_back=1):
        dataX, dataY = [], []
        dataZ=[]
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 1]
            dataX.append(a)
            b = dataset[i + look_back, 0]
            dataZ.append(b)
            dataY.append(dataset[i + look_back, 1])
        return numpy.array(dataX), numpy.array(dataY),numpy.array(dataZ)              

In [120]:
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
# dataframe = read_csv('w_d_v.csv', usecols=[7], engine='python', skipfooter=3)
dataframe = read_csv('t6192.csv', usecols=[8,0], engine='python',dtype=np.int32,skiprows=-1,header=None,skipfooter=3)
pattern = read_csv('t6192.csv', usecols=[7], engine='python',dtype=np.int32,skiprows=-1,header=None,skipfooter=3)
Matrix = read_csv('matrix621.csv', usecols=[2,3,4,5,6,7,8,9,10,11,12,13], engine='python',header=None)
all_data = read_csv('all_data.csv', usecols=[7], engine='python', skipfooter=3)
dataset = dataframe.values
Matrix = Matrix.values
pattern=pattern.values
allData=all_data.values
Matrix=np.append([[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]],Matrix,axis=0)

In [122]:
look_back = 3

trainX, trainY, trainZ = create_dataset2(dataset, look_back)
AllX, AllY = create_dataset(allData, look_back)
patternX, patternY = create_dataset(pattern, look_back)

trainY=numpy.reshape(trainY,(trainY.shape[0],-1))
AllY=numpy.reshape(AllY,(AllY.shape[0],-1))

In [123]:
encX = OneHotEncoder()
encX.fit(trainX)
encY = OneHotEncoder()
encY.fit(trainY)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [124]:
trainX_one=encX.transform(trainX).toarray()
train_X=numpy.reshape(trainX_one,(trainX_one.shape[0],look_back,-1))
train_Y=encY.transform(trainY).toarray()

#还没能直接拆分，其他维度没有做对应
a_train, a_test, b_train, b_test = train_test_split(train_X, train_Y, test_size=0.1, random_state=42)

In [125]:
emdedding_size=Matrix.shape[1] #
vo_len=look_back #
vocab_size=Matrix.shape[0] #

In [126]:

print("------------------------")
print("emdedding_size:")
print(emdedding_size)
print("------------------------")

print("vocab_length:")
print(vo_len)
print("------------------------")

print("vocab_size:")
print(vocab_size)
print("------------------------")


------------------------
emdedding_size:
12
------------------------
vocab_length:
3
------------------------
vocab_size:
201
------------------------


In [127]:
a_train=trainX.reshape(-1,3,1)
b_train=train_Y
k=trainZ
pretrained_weights=Matrix
LSTM_size=32

In [128]:
print("------------------------")
print("in size:")
print(a_train.shape)
print("------------------------")
print("out size:")
print(b_train.shape)
print("------------------------")
print("user size:")
print(k.shape)
print("------------------------")

------------------------
in size:
(32607, 3, 1)
------------------------
out size:
(32607, 191)
------------------------
user size:
(32607,)
------------------------


In [129]:
print("------------------------")
print("input encode example1:")
print(train_X[0])
print("------------------------")

print("input encode example2:")
for x in a_train[0]:
    print(pretrained_weights[x])
print("------------------------")

print("input decode example:")
print(a_train[0])
print("------------------------")

print("output encode example:")
print(b_train[0])
print("------------------------")

print("output decode example:")
print(trainY[0])
print("------------------------")

print("user_id example:")
print(k[0])
print("------------------------")

------------------------
input encode example1:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [131]:
a_train=a_train.reshape(-1,3)

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.models import Model


input_pattern = Input(shape=(3, ),name="input_pattern")
input_id = Input(shape=(1,),name="input_id")

em = Embedding(input_dim=vocab_size, output_dim=emdedding_size,input_length=vo_len)(input_pattern)
lstm_out = LSTM(LSTM_size)(em)
lstm_out = Dropout(0.2)(lstm_out)

x = keras.layers.concatenate([lstm_out, input_id])
x=Dense(250,activation='relu',name="C")(x)
x=Dropout(0.2)(x)
x=Dense(b_train.shape[1],activation='softmax',name='x')(x)


model = Model(inputs=[input_pattern,input_id], outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print(model.summary()) # Summarize Model
plot_model(model, to_file='t_lstm.png',show_shapes=True)
history_nopre = model.fit([a_train,k], b_train, epochs=100, batch_size=16, verbose=2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pattern (InputLayer)      (None, 3)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 3, 12)        2412        input_pattern[0][0]              
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           5760        embedding_1[0][0]                
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 32)           0           lstm_1[0][0]                     
__________________________________________________________________________________________________
input_id (

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.models import Model



input_pattern = Input(shape=(3, 1),name="input_pattern")

lstm_out = LSTM(units=emdedding_size,input_shape=(3, 1))(input_pattern)
lstm_out = Dropout(0.2)(lstm_out)

x=Dense(250,activation='relu',name="C")(lstm_out)
x=Dropout(0.2)(x)
x=Dense(180,activation='softmax')(x)


model = Model(inputs=input_pattern, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# print(model.summary()) # Summarize Model
plot_model(model, to_file='t_lstm_test.png',show_shapes=True)
history_nopre = model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2)

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.models import Model

emdedding_size=12 #
vo_len=3 #
vocab_size=11000 #
a_train=trainX.reshape(-1,3)
b_train=train_Y
k=trainZ
pretrained_weights=Matrix

input_pattern = Input(shape=(3, ),name="input_pattern")
em = Embedding(input_dim=vocab_size, output_dim=emdedding_size,input_length=vo_len)(input_pattern)
lstm_out = LSTM(units=emdedding_size)(em)
lstm_out = Dropout(0.2)(lstm_out)

x=Dense(250,activation='relu',name="C")(lstm_out)
x=Dropout(0.2)(x)
x=Dense(180,activation='softmax')(x)


model = Model(inputs=input_pattern, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print(model.summary()) # Summarize Model
plot_model(model, to_file='t_lstm_test.png',show_shapes=True)
history_withpre = model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2)

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.models import Model

emdedding_size=12 #
vo_len=3 #
vocab_size=11000 #
a_train=patternX
b_train=train_Y
k=trainZ
pretrained_weights=Matrix

input_pattern = Input(shape=(3, ),name="input_pattern")
em = Embedding(input_dim=vocab_size, output_dim=emdedding_size,input_length=vo_len, weights=[pretrained_weights])(input_pattern)
lstm_out = LSTM(units=emdedding_size)(em)
lstm_out = Dropout(0.2)(lstm_out)

x=Dense(250,activation='relu',name="C")(lstm_out)
x=Dropout(0.2)(x)
x=Dense(180,activation='softmax')(x)


model = Model(inputs=input_pattern, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print(model.summary()) # Summarize Model
plot_model(model, to_file='t_lstm_test.png',show_shapes=True)
history_withpre2 = model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2)

plot_model(model, to_file='t_lstm_test.png',show_shapes=True)
history_nopre = model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2)

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.models import Model

emdedding_size=12 #
vo_len=3 #
vocab_size=11000 #
a_train=trainX.reshape(-1,3)
b_train=train_Y
k=trainZ
pretrained_weights=Matrix

input_pattern = Input(shape=(3, ),name="input_pattern")
em = Embedding(input_dim=vocab_size, output_dim=emdedding_size,input_length=vo_len, weights=[pretrained_weights])(input_pattern)
lstm_out = LSTM(units=emdedding_size)(em)
lstm_out = Dropout(0.2)(lstm_out)

x=Dense(250,activation='relu',name="C")(lstm_out)
x=Dropout(0.2)(x)
x=Dense(180,activation='softmax')(x)


model = Model(inputs=input_pattern, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print(model.summary()) # Summarize Model
plot_model(model, to_file='t_lstm_test.png',show_shapes=True)
history_withpre2 = model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2)

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.models import Model

emdedding_size=12 #
vo_len=3 #
vocab_size=11000 #
a_train=patternX.reshape(-1,3,1)
b_train=train_Y
k=trainZ
pretrained_weights=Matrix

input_pattern = Input(shape=(3, 1),name="input_pattern")
lstm_out = LSTM(units=64)(input_pattern)
lstm_out = Dropout(0.2)(lstm_out)

x=Dense(250,activation='relu',name="C")(lstm_out)
x=Dropout(0.2)(x)
x=Dense(180,activation='softmax')(x)


model = Model(inputs=input_pattern, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print(model.summary()) # Summarize Model
plot_model(model, to_file='t_lstm_test.png',show_shapes=True)
history_withpre2 = model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2)

plot_model(model, to_file='t_lstm_test.png',show_shapes=True)
history1 = model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2)

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.models import Model

input_pattern = Input(shape=(3, a_train.shape[2]),name="input_pattern")

lstm_out = LSTM(512,input_shape=(3, a_train.shape[2]))(input_pattern)
# lstm_out = LSTM(512,return_sequences=True,input_shape=(3, a_train.shape[2]))(input_pattern)
# lstm_out = LSTM(300)(lstm_out)
lstm_out = Dropout(0.2)(lstm_out)

x=Dense(250,activation='relu',name="C")(lstm_out)
x=Dropout(0.2)(x)
x=Dense(a_train.shape[2],activation='softmax')(x)

model = Model(inputs=input_pattern, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print(model.summary()) # Summarize Model
plot_model(model, to_file='t_lstm_test.png',show_shapes=True)

In [ ]:
history = model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2, validation_data=(a_test, b_test))

In [ ]:
print(history.history.keys())

In [ ]:
fig = plt.figure()
plt.plot(history.history['acc'])
plt.plot(history1.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['1-lstm', '2-lstm'], loc='upper left')

In [ ]:
fig = plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower left')

In [ ]:
train_X=train_X.reshape(-1,200)
train_Y.reshape(-1,200)
train_X.shape

In [ ]:
train_Y.shape

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.models import Model

a_train=train_X
b_train=train_Y
k=trainZ

input_pattern = Input(shape=(3, a_train.shape[2]),name="input_pattern")
input_id = Input(shape=(1,),name="input_id")

lstm_out = LSTM(250,input_shape=(3, a_train.shape[2]))(input_pattern)
lstm_out = Dropout(0.2)(lstm_out)

x = keras.layers.concatenate([lstm_out, input_id])
x=Dense(250,activation='relu',name="C")(x)
x=Dropout(0.2)(x)
x=Dense(a_train.shape[2],activation='softmax',name='x')(x)


model = Model(inputs=[input_pattern,input_id], outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print(model.summary()) # Summarize Model
plot_model(model, to_file='t_lstm.png',show_shapes=True)

In [ ]:
k=np.zeros(a_train.shape[0],dtype=np.int16)
k=k.reshape(-1,1)

k1=np.zeros(train_X.shape[0],dtype=np.int16)
k1=k1.reshape(-1,1)

In [ ]:
history = model.fit({'input_pattern': a_train, 'input_id' : k}, {'x': b_train}, epochs=100, batch_size=64, verbose=2)

In [ ]:
fig = plt.figure()
Accuracy=[42.00,47.15 ,48.36, 49.35,47.42, 50.82, 52.31,56.93 ,57.15 ]
x2=(20,30,40,50,60,70,80,90,100)
plt.plot(x2,Accuracy)
x1=range(0,100)
plt.plot(x1,history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['100% total_data Yu','100% total_data Mine'], loc='upper left')



In [ ]:
fig = plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['90% train_data', '100% total_data'], loc='upper left')

model.fit(a_train, b_train, epochs=100, batch_size=16, verbose=2, validation_data=(a_test, b_test))

In [ ]:
model.evaluate(train_X, train_Y, batch_size=64, verbose=2, sample_weight=None)

In [ ]:
trainPredict = model.predict(train_X)
D=np.argmax(train_Y,axis = 1)
E=np.argmax(trainPredict,axis = 1)

In [ ]:
print(D)
print(E)

In [ ]:
A=0 #total number of right
for i,t in enumerate(E):
    if D[i]==t :
        A=A+1
print(A/D.shape[0])